In [1]:
import numpy as np
import os
import utils
from extractDigitFeatures import extractDigitFeatures

In [2]:
data = utils.loadmat('../data/digits-normal.mat') 
features = extractDigitFeatures(data['x'], 'hog')
#model = trainModel(features[:, data['set']==trainSet], data['y'][data['set']==trainSet])
trainSet, testSet = 1, 2
param = {}
param['lambda'], param['maxiter'], param['eta'] = 0.01, 200,  .01
# trainModel : x = features[data['set']==trainSet, :], y = data['y'][data['set']==trainSet]
x_train = features[:, data['set']==trainSet]
y_train = data['y'][data['set']==trainSet]

x_val = features[:, data['set']==3]
y_val = data['y'][data['set']==3]

x_test = features[:, data['set']==testSet]
y_test = data['y'][data['set']==testSet]

# multiclassLRTrain : x, y, param
classLabels = np.unique(y_train)
numClass = classLabels.shape[0]
numFeats = x_train.shape[0]
numData = x_train.shape[1]

# Initialize weights randomly (Implement gradient descent)
#model = {}
#model['w'] = np.random.randn(numClass, numFeats)*0.01
#model['classLabels'] = classLabels

In [3]:
class LogisticRegression(object):
    def __init__(self, eta, lambda_, maxEpochs, numClass, debug=False, maxTol=10):
        self.lr = eta
        self.lambda_ = lambda_
        self.max_epochs = maxEpochs
        self.debug = debug
        self.numClass = numClass
        self.bias = np.random.uniform(low=-0.01, high=.01)
        self.maxTol = maxTol
        self.lastAccuracy = -1
        
    def softmax(self, X):
        exp = np.exp(X - np.max(X))
        return exp / np.array([np.sum(exp, axis=1)]).T
    
    def accuracy(self, X, y):
        y_pred = np.argmax(self.softmax(np.dot(X.T, self.weights)+self.bias), axis=1)
        return np.sum([y_pred[i] == y[i] for i in range(y.size)]) / y.size
        
    def fit(self, X, y, X_val, y_val):
        self.weights = np.random.uniform(low=-0.01, high=.01, size=(X.shape[0], numClass))
        y_categorical = np.eye(self.numClass)[y] 
        curTol = 0
        
        for itr in range(self.max_epochs):
            prediction = self.softmax(np.dot(X.T, self.weights))
            error = y_categorical - prediction
            gradient = np.dot(X, error)
            self.weights += self.lr * (gradient - (self.lambda_ * self.weights ))
            self.bias -= self.lr * np.sum(error)
            
            
            curr_accuracy = self.accuracy(X_val, y_val)
            if curr_accuracy != self.lastAccuracy:
                self.lastAccuracy = curr_accuracy
            else:
                curTol += 1
                if curTol >= self.maxTol:
                    return self
            
            if itr % 50 == 0 and self.debug: 
                print ("Accuracy = {}".format(curr_accuracy)
                       
        return self
    
    def predict(self, X, y):
        return self.accuracy(X, y)

In [5]:
learning_rate = [.0001, .0005, .001, .005, .01, .05, .1, .5]
lambda_val = [.001, .005, .01, .05, .1, .5, 1, 2]
for lr in learning_rate:
    for lambda_ in lambda_val:
        model = LogisticRegression(lr, lambda_, 3000, numClass)
        model = model.fit(x_train, y_train, x_val, y_val)
        accuracy = model.predict(x_test, y_test)
        print ("Accuracy = {} for Lr = {} and Lambda = {}".format(accuracy, lr, lambda_))

Accuracy = 0.888 for Lr = 0.0001 and Lambda = 0.001
Accuracy = 0.888 for Lr = 0.0001 and Lambda = 0.005
Accuracy = 0.888 for Lr = 0.0001 and Lambda = 0.01
Accuracy = 0.89 for Lr = 0.0001 and Lambda = 0.05
Accuracy = 0.888 for Lr = 0.0001 and Lambda = 0.1
Accuracy = 0.888 for Lr = 0.0001 and Lambda = 0.5
Accuracy = 0.888 for Lr = 0.0001 and Lambda = 1
Accuracy = 0.89 for Lr = 0.0001 and Lambda = 2
Accuracy = 0.894 for Lr = 0.0005 and Lambda = 0.001
Accuracy = 0.894 for Lr = 0.0005 and Lambda = 0.005
Accuracy = 0.894 for Lr = 0.0005 and Lambda = 0.01
Accuracy = 0.894 for Lr = 0.0005 and Lambda = 0.05
Accuracy = 0.894 for Lr = 0.0005 and Lambda = 0.1
Accuracy = 0.896 for Lr = 0.0005 and Lambda = 0.5
Accuracy = 0.894 for Lr = 0.0005 and Lambda = 1
Accuracy = 0.89 for Lr = 0.0005 and Lambda = 2
Accuracy = 0.902 for Lr = 0.001 and Lambda = 0.001
Accuracy = 0.902 for Lr = 0.001 and Lambda = 0.005
Accuracy = 0.9 for Lr = 0.001 and Lambda = 0.01
Accuracy = 0.902 for Lr = 0.001 and Lambda = 0.05